In [9]:
# Imports

import pandas as pd
import qgrid
from sklearn.linear_model import LinearRegression
import os

from tqdm.notebook import tqdm
from highcharts import Highstock

from data_management.database import DataBase

In [17]:
db = DataBase()

In [3]:
# Calculation functions

def performance(series):
    X = series.index.values.reshape(-1,1)
    y = series.values.reshape(-1,1)

    regressor = LinearRegression()  
    regressor.fit(X, y)
    
    perf = regressor.coef_[0][0]
    yearly = ((perf * 250) / series.iloc[-1]) *100
    return yearly

In [4]:
# Get instruments list

                        path = '/Users/martin/Google Drive/data/screener/'
                        files = os.listdir(path)
                        dfi = pd.DataFrame()
                        dfi['Instrument'] = files[:]
# Todo: Get instruments from databse

In [18]:
# Get contracts
query = """SELECT contract_id, symbol, name, exchange
            FROM contracts"""

conn = db.connect()
cur = conn.cursor()

dfc = pd.read_sql_query(query, conn)

conn.commit()
cur.close()
db.disconnect(conn)

In [19]:
dfc

,contract_id,symbol,name,exchange
0,1,BNQF,Collateralized ETC on RICI Enhanced Gas Oil TR...,FWB
1,2,CNB,Lyxor Euro Corporate Bond Ex Financials UCITS ETF,FWB
2,3,UEF5,UBS ETF - MSCI Emerging Markets Socially Respo...,FWB
3,5,FLOT,Lyxor Barclays Floating Rate Euro 0-7Y UCITS ETF,FWB
4,7,ETLH,L&G Ecommerce Logistics UCITS ETF,FWB
...,...,...,...,...
4339,5081,VMIG,Vanguard FTSE 250 UCITS ETF,LSEETF
4340,5085,VWRD,Vanguard FTSE All-World UCITS ETF,LSEETF
4341,5086,EEDM,iShares MSCI EM ESG Enhanced UCITS ETF,LSEETF
4342,5087,VUKG,Vanguard FTSE 100 UCITS ETF,LSEETF


In [5]:
# Add calculations to instruments

for contract in tqdm(dfi['Instrument']):
    df = pd.read_csv(path + inst)
    if len(df) == 0:
        continue
    dfi.loc[dfi['Instrument']==inst, 'Perf_3m'] = performance(df.iloc[-66:]['close'])
    dfi.loc[dfi['Instrument']==inst, 'Perf_1y'] = performance(df.iloc[-250:]['close'])
    dfi.loc[dfi['Instrument']==inst, 'Perf_2y'] = performance(df.iloc[-500:]['close'])

In [6]:
# Add calculations to instruments, multiprocessing

from multiprocessing import Pool

def add_calc(inst):
    df = pd.read_csv(path + inst)
    if len(df) == 0:
        return
    dfi.loc[dfi['Instrument']==inst, 'Perf_3m'] = performance(df.iloc[-66:]['close'])
    dfi.loc[dfi['Instrument']==inst, 'Perf_1y'] = performance(df.iloc[-250:]['close'])
    dfi.loc[dfi['Instrument']==inst, 'Perf_2y'] = performance(df.iloc[-500:]['close'])
    
instrument_list = dfi['Instrument'][:]
with Pool(8) as p:
    p.map(add_calc, instrument_list)

In [20]:
# Build QGrid widget

qgw = qgrid.show_grid(dfc, grid_options={'maxVisibleRows': 30})

In [21]:
# QGrid handlers

def handle_selection_changed(event, widget):
    line_no = event['new'][0]
    build_chart(line_no)
    
qgw.on('selection_changed', handle_selection_changed)


def handle_data_changed(event, widget):
    global dfc
    dfc = qgw.get_changed_df()

qgw.on('filter_changed', handle_data_changed)
qgw.on('sort_changed', handle_data_changed)

In [22]:
# Chart builder

def build_chart(line_no):
    # filename = dfi.iloc[line_no,]['Instrument']
    # filepath = '/Users/martin/Google Drive/data/screener/'
    # df = pd.read_csv(filepath + filename)
    # df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
    # df['date'] = (df['date'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1ms')

    # Get quotes
    contract_id = dfc.iloc[line_no,]['contract_id']
    query = f"""SELECT date, open, high, low, close
                FROM quotes
                WHERE contract_id = {contract_id}
                ORDER BY date ASC;"""

    conn = db.connect()
    cur = conn.cursor()
    df = pd.read_sql_query(query, conn)
    conn.commit()
    cur.close()
    db.disconnect(conn)

    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

    chart = Highstock()
    data = df[['date','open','high','low','close']].values.tolist()
    chart.add_data_set(data, series_type='ohlc', name=str(contract_id))
    # todo: add moving average

    options = {
        'chart': {
            'animation': False,
            'height': 800,
            'marginLeft': 50
        },
        'rangeSelector': {
            'selected': 5,
            'buttons': [{
                'type': 'month',
                'count': 1,
                'text': '1m'
            }, {
                'type': 'month',
                'count': 3,
                'text': '3m'
            }, {
                'type': 'month',
                'count': 6,
                'text': '6m'
            }, {
                'type': 'ytd',
                'text': 'YTD'
            }, {
                'type': 'year',
                'count': 1,
                'text': '1y'
            }, {
                'type': 'year',
                'count': 2,
                'text': '2y'
            }, {
                'type': 'all',
                'text': 'All'
            }]
        },
        'navigator': {
            'height': 100
        },
        'title': {
            'text': str(contract_id)
        },
        'plotOptions': {
            'series': {
                'animation': False
            }
        },
        'yAxis': {
            'type': 'logarithmic'
        }
    }
    chart.set_dict_options(options)
    chart.save_file('./screener/website/index')

In [23]:
##### Show QGrid widget

qgw

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…